In [ ]:
kaggle = True
should_train = False
input_dir = '../input/' if kaggle else './'
base_dir = '../input/imet-2020-fgvc7/' if kaggle else '/macierz/home/s165138/UGWK/'
model_path_for_test = f"../input/test222/best-model(1).pt" if kaggle else './best-model.pt'
train_root = base_dir + 'train'
number_of_classes = 3474
num_workers = 4
batch_size = 64

In [ ]:
from collections import defaultdict, Counter
from typing import Callable, List, Dict
from pathlib import Path
from itertools import islice
from functools import partial
from PIL import Image
from sklearn.metrics import fbeta_score
from sklearn.exceptions import UndefinedMetricWarning
from torch import nn, cuda
from torch.nn import functional as F
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Normalize, Compose, Resize, CenterCrop, RandomCrop, RandomHorizontalFlip
import pandas as pd
import numpy as np
import torchvision.models as M
import os
import random
import cv2
import torch
import math
import json
import shutil
import warnings
import torch

In [ ]:
def make_folds(n_folds: int, labels_file_path: str) -> pd.DataFrame:
    df = pd.read_csv(labels_file_path)
    cls_counts = Counter(cls for classes in df['attribute_ids'].str.split() for cls in classes)
    fold_cls_counts = defaultdict(int)
    folds = [-1] * len(df)
    for item in df.sample(frac=1, random_state=42).itertuples():
        cls = min(item.attribute_ids.split(), key=lambda cls: cls_counts[cls])
        fold_counts = [(f, fold_cls_counts[f, cls]) for f in range(n_folds)]
        min_count = min([count for _, count in fold_counts])
        random.seed(item.Index)
        fold = random.choice([f for f, count in fold_counts
                              if count == min_count])
        folds[item.Index] = fold
        for cls in item.attribute_ids.split():
            fold_cls_counts[fold, cls] += 1
    df['fold'] = folds
    return df

In [ ]:
def save_folds(n_folds: int, labels_file_path: str, save_path: str):
    df = make_folds(n_folds=5, labels_file_path=base_dir + 'train.csv')
    df.to_csv(save_path, index=None)

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, root_dir: Path, number_of_classes: int, df: pd.DataFrame, image_transform: Callable, tensor_transform: Callable):
        self.root_dir = root_dir
        self.df = df
        self.image_transform = image_transform
        self.tensor_transform = tensor_transform
        self.number_of_classes = number_of_classes

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        item = self.df.iloc[idx]
        image = load_transform_image(item.id, self.root_dir, self.image_transform, self.tensor_transform)
        target = torch.zeros(self.number_of_classes)
        
        for attribute in item.attribute_ids.split():
            target[int(attribute)] = 1

        return image, target

class PredictImageDataset(Dataset):
    def __init__(self, root_dir: Path, image_transform, tensor_transform):
        self.root_dir = root_dir
        self.files = os.listdir(root_dir)
        self.tensor_transform = tensor_transform
        self.image_transform = image_transform
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.files[idx])
        img_name = os.path.splitext(self.files[idx])[0]
        image = load_transform_image(img_name, self.root_dir, self.image_transform, self.tensor_transform)
        return image
    
    def filenames(self):
        return [os.path.splitext(x)[0] for x in self.files]

In [ ]:
def load_transform_image(item_id, root_dir: Path, image_transform: Callable, tensor_transform: Callable):
    image = load_image(item_id, root_dir)
    image = image_transform(image)
    return tensor_transform(image)


def load_image(item_id, root_dir: Path) -> Image.Image:
    path = root_dir + f'/{item_id}.png'
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return Image.fromarray(image)

In [ ]:
def make_loader(df: pd.DataFrame, image_transform: Callable, tensor_transform: Callable) -> DataLoader:
    dataset = TrainDataset(train_root, number_of_classes, df, image_transform, tensor_transform)
    return DataLoader(dataset, shuffle=True, batch_size=batch_size, num_workers=num_workers)

In [ ]:
class AvgPool(nn.Module):
    def forward(self, x):
        return F.avg_pool2d(x, x.shape[2:])


def create_net(net_cls, pretrained: bool):
    if pretrained:
        net = net_cls()
        model_name = net_cls.__name__
        print(model_name)
        weights_path = input_dir + f'/d/pytorch/resnet50/resnet50.pth'
        net.load_state_dict(torch.load(weights_path))
    else:
        net = net_cls(pretrained=pretrained)
    return net


class ResNet(nn.Module):
    def __init__(self, num_classes, pretrained=False, net_cls=M.resnet50, dropout=False):
        super().__init__()
        self.net = create_net(net_cls, pretrained=pretrained)
        self.net.avgpool = AvgPool()
        if dropout:
            self.net.fc = nn.Sequential(
                nn.Dropout(p=0.2),
                nn.Linear(self.net.fc.in_features, num_classes),
            )
        else:
            self.net.fc = nn.Linear(self.net.fc.in_features, num_classes)

    def forward(self, x):
        return self.net(x)

In [ ]:
def load_model(model: nn.Module, path: Path) -> Dict:
    state = torch.load(str(path))
    model.load_state_dict(state['model'])
    epoch = state['epoch']
    step = state['step']
    print(f'Loaded model from epoch {epoch}, step {step}')
    return state

In [ ]:
def reduce_loss(loss):
    return loss.sum() / loss.shape[0]

In [ ]:
def binarize_prediction(probabilities, threshold: float, argsorted=None,
                        min_labels=1, max_labels=10):
    """ Return matrix of 0/1 predictions, same shape as probabilities.
    """
    assert probabilities.shape[1] == number_of_classes
    if argsorted is None:
        argsorted = probabilities.argsort(axis=1)
    max_mask = make_mask(argsorted, max_labels)
    min_mask = make_mask(argsorted, min_labels)
    prob_mask = probabilities > threshold
    return (max_mask & prob_mask) | min_mask


def make_mask(argsorted, top_n: int):
    mask = np.zeros_like(argsorted, dtype=np.uint8)
    col_indices = argsorted[:, -top_n:].reshape(-1)
    row_indices = [i // top_n for i in range(len(col_indices))]
    mask[row_indices, col_indices] = 1
    return mask


In [ ]:
def validation(
        model: nn.Module, criterion, valid_loader, use_cuda,
        ) -> Dict[str, float]:
    model.eval()
    all_losses, all_predictions, all_targets = [], [], []
    with torch.no_grad():
        for inputs, targets in valid_loader:
            all_targets.append(targets.numpy().copy())
            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            all_losses.append(reduce_loss(loss).item())
            predictions = torch.sigmoid(outputs)
            all_predictions.append(predictions.cpu().numpy())
    all_predictions = np.concatenate(all_predictions)
    all_targets = np.concatenate(all_targets)

    def get_score(y_pred):
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', category=UndefinedMetricWarning)
            return fbeta_score(
                all_targets, y_pred, beta=2, average='samples')

    metrics = {}
    argsorted = all_predictions.argsort(axis=1)
    for threshold in [0.10, 0.20]:
        metrics[f'valid_f2_th_{threshold:.2f}'] = get_score(
            binarize_prediction(all_predictions, threshold, argsorted))
    metrics['valid_loss'] = np.mean(all_losses)
    print(' | '.join(f'{k} {v:.3f}' for k, v in sorted(
        metrics.items(), key=lambda kv: -kv[1])))

    return metrics

In [ ]:
def train(model: nn.Module, criterion, *, params,
          train_loader, valid_loader, init_optimizer, use_cuda,
          n_epochs=None, patience=2, max_lr_changes=2) -> bool:
    
    lr = 1e-4
    n_epochs = 100
    params = list(params)
    optimizer = init_optimizer(params, lr)

    model_path = 'model.pt'
    best_model_path = 'best-model.pt'
    #we could make it more automatic...
    uptrain = False
    if uptrain:
        state = load_model(model, model_path)
        epoch = state['epoch']
        step = state['step']
        best_valid_loss = state['best_valid_loss']
    else:
        epoch = 1
        step = 0
        best_valid_loss = float('inf')
    lr_changes = 0

    save = lambda ep: torch.save({
        'model': model.state_dict(),
        'epoch': ep,
        'step': step,
        'best_valid_loss': best_valid_loss
    }, str(model_path))

    report_each = 100
    valid_losses = []
    lr_reset_epoch = epoch
    for epoch in range(epoch, n_epochs + 1):
        model.train()
        #tq = tqdm.tqdm(total=(len(train_loader) * batch_size))
        #tq.set_description(f'Epoch {epoch}, lr {lr}')
        losses = []
        tl = train_loader
        mean_loss = 0
        
        epoch_progress = 0
        for i, (inputs, targets) in enumerate(tl):
            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)
            loss = reduce_loss(criterion(outputs, targets))
            batch_size = inputs.size(0)
            (batch_size * loss).backward()
            if (i + 1) % 1 == 0:
                optimizer.step()
                optimizer.zero_grad()
                step += 1
            #tq.update(batch_size)
            losses.append(loss.item())
            mean_loss = np.mean(losses[-report_each:])
            epoch_progress += len(inputs)
            print(f'mean_loss: {mean_loss:.3f} epoch progess {epoch_progress}/113694')
            #tq.set_postfix(loss=f'{mean_loss:.3f}')
        #tq.close()
        save(epoch + 1)
        valid_metrics = validation(model, criterion, valid_loader, use_cuda)
        
        valid_loss = valid_metrics['valid_loss']
        valid_losses.append(valid_loss)
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            shutil.copy(str(model_path), str(best_model_path))
        elif (patience and epoch - lr_reset_epoch > patience and
              min(valid_losses[-patience:]) > best_valid_loss):
            lr_changes +=1
            if lr_changes > max_lr_changes:
                break
            lr /= 5
            print(f'lr updated to {lr}')
            lr_reset_epoch = epoch
            optimizer = init_optimizer(params, lr)
    return True

In [ ]:
class RandomErasing(object):
    def __init__(self, EPSILON = 0.5, sl = 0.02, sh = 0.4, r1 = 0.3, mean=[0.4914, 0.4822, 0.4465]):
        self.EPSILON = EPSILON
        self.mean = mean
        self.sl = sl
        self.sh = sh
        self.r1 = r1
       
    def __call__(self, img):

        if random.uniform(0, 1) > self.EPSILON:
            return img

        for attempt in range(100):
            area = img.size()[1] * img.size()[2]
       
            target_area = random.uniform(self.sl, self.sh) * area
            aspect_ratio = random.uniform(self.r1, 1/self.r1)

            h = int(round(math.sqrt(target_area * aspect_ratio)))
            w = int(round(math.sqrt(target_area / aspect_ratio)))

            if w < img.size()[2] and h < img.size()[1]:
                x1 = random.randint(0, img.size()[1] - h)
                y1 = random.randint(0, img.size()[2] - w)
                if img.size()[0] == 3:
                    #img[0, x1:x1+h, y1:y1+w] = random.uniform(0, 1)
                    #img[1, x1:x1+h, y1:y1+w] = random.uniform(0, 1)
                    #img[2, x1:x1+h, y1:y1+w] = random.uniform(0, 1)
                    img[0, x1:x1+h, y1:y1+w] = self.mean[0]
                    img[1, x1:x1+h, y1:y1+w] = self.mean[1]
                    img[2, x1:x1+h, y1:y1+w] = self.mean[2]
                    #img[:, x1:x1+h, y1:y1+w] = torch.from_numpy(np.random.rand(3, h, w))
                else:
                    img[0, x1:x1+h, y1:y1+w] = self.mean[1]
                    # img[0, x1:x1+h, y1:y1+w] = torch.from_numpy(np.random.rand(1, h, w))
                return img

        return img

In [ ]:
import torch
import random
import torchvision.transforms as transforms

from PIL import Image

def brightness(img, delta):
    if random.random() < 0.5:
        img = transforms.ColorJitter(brightness=delta)(img)
    return img
def contrast(img, delta):
    if random.random() < 0.5:
        img = transforms.ColorJitter(contrast=delta)(img)
    return img
def saturation(img, delta):
    if random.random() < 0.5:
        img = transforms.ColorJitter(saturation=delta)(img)
    return img
def hue(img, delta):
    if random.random() < 0.5:
        img = transforms.ColorJitter(hue=delta)(img)
    return img

class RandomDistort(object):
    def __init__(self,
        brightness_delta=32/255.,
        contrast_delta=0.5,
        saturation_delta=0.5,
        hue_delta=18/255.):
        
        self.EPSILON = 0.5
        self.brightness_delta = brightness_delta
        self.contrast_delta = contrast_delta
        self.saturation_delta = saturation_delta
        self.hue_delta = hue_delta
        
    def __call__(self, img):
        if random.uniform(0, 1) > self.EPSILON:
            return img
        
        img = brightness(img, self.brightness_delta)
        if random.random() < 0.5:
            img = contrast(img, self.contrast_delta)
            img = saturation(img, self.saturation_delta)
            img = hue(img, self.hue_delta)
        else:
            img = saturation(img, self.saturation_delta)
            img = hue(img, self.hue_delta)
            img = contrast(img, self.contrast_delta)
        return img
        

    
        

In [ ]:


train_transform = Compose(
[
    Resize((288,288)),
    #RandomCrop(288),
    RandomHorizontalFlip(),
])

test_transform = Compose(
[
    Resize((288,288)),
    #RandomCrop(256),
    RandomHorizontalFlip(),
])

eval_transform = Compose(
[
    Resize((288,288))
    #CenterCrop(288)
])

train_tensor_transform = Compose(
[
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_tensor_transform = Compose(
[
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
def prepare_folds():
    folds = make_folds(n_folds=5, labels_file_path=base_dir + 'train.csv')

    #save_path = '../input/imet2002folds/folds.csv'
    #save_folds(n_folds=5, labels_file_path=base_dir + 'train.csv', save_path=save_path)
    #read pre-made
    #pd.read_csv(save_path)
    return folds

In [ ]:
def start_train():
    folds = prepare_folds()
    train_fold = folds[folds['fold'] != 0]
    valid_fold = folds[folds['fold'] == 0]

    train_loader = make_loader(train_fold, train_transform, train_tensor_transform)
    valid_loader = make_loader(valid_fold, test_transform, test_tensor_transform)

    criterion = nn.BCEWithLogitsLoss(reduction='none')

    model = ResNet(num_classes=number_of_classes, pretrained=True, net_cls=M.resnet50)

    use_cuda = cuda.is_available()
    if use_cuda:
        model = model.cuda()

    train(
        params=model.parameters(),
        model=model,
        criterion=criterion,
        train_loader=train_loader,
        valid_loader=valid_loader,
        patience=4,
        init_optimizer=lambda params, lr: Adam(params, lr),
        use_cuda=use_cuda)

In [ ]:
def predict(model, test_dataset, batch_size):
        loader = torch.utils.data.DataLoader(test_dataset, 
                                 batch_size=batch_size, 
                                 shuffle=False)
        
        preds = np.zeros((len(test_dataset), 1103))
        model.cuda()
        model.eval()
        temp = np.zeros_like(preds)
         
        model_result = []
        
        it = 0
        for i_batch in loader:
            print(it)
            inputs = i_batch.cuda()
            with torch.no_grad():
                model_batch_result = model(inputs)
                model_result.extend(model_batch_result.cpu().numpy())
            it = it + batch_size
        return model_result

In [ ]:
def save_predictions(preds, test_dataset):
    preds_fin = (np.array(preds) > 0.2).astype(int)
    prediction = []
    for i in range(preds_fin.shape[0]):
        pred1 = np.argwhere(preds_fin[i] == 1.0).reshape(-1).tolist()
        pred_str = " ".join(list(map(str, pred1)))
        prediction.append(pred_str)

    sample_path = os.path.join(base_dir, "sample_submission.csv")
    sample = pd.read_csv(sample_path)
    sample.head()
    sample.id = test_dataset.filenames()
    sample.attribute_ids = prediction
    sample.to_csv("submission.csv", index=False)
    sample.head()

In [ ]:
def test():
    batch_size = 200
    test_img_dir = os.path.join(base_dir, "test")
    test_dataset = PredictImageDataset(test_img_dir, eval_transform, test_tensor_transform)
    new_model = ResNet(num_classes=number_of_classes)
    load_model(new_model, model_path_for_test)
    new_model.eval()
    preds = predict(new_model, test_dataset, batch_size)
    save_predictions(preds, test_dataset)

In [ ]:
if should_train:
    start_train()
else:
    test()